In [11]:
import sys
import collections

base_file = "../data/u1.base"
test_file = "../data/u1.test" #[user_id]\t[item_id]\t[rating]\t[time_stamp]\n
output_file = "../data/u"+"1"+".base_prediction.txt" #[user_id]\t[item_id]\t[rating]\n

neighbor_number = 10

In [12]:
with open(base_file) as base :
    data_set = base.read().split()
    data_set = list(map(int, data_set))
    data_set = [data_set[i:i+4] for i in range(0, len(data_set), 4)]

In [13]:
data_set_per_user = {}
for i in data_set :
    if i[0] in data_set_per_user.keys() :
        data_set_per_user[i[0]].append(([i[1], i[2], i[3]]))
    else :
        data_set_per_user[i[0]] = [([i[1], i[2], i[3]])]

In [14]:
def get_neighbor (data_per_user) :
    intersection_per_user = {} # common # : user pair

    users = list(data_per_user.keys())

    count = 0
    for i in users :
        for j in users :
            if users.index(i) >= users.index(j) :
                continue

            """
            common = get_common_item(data_per_user[i], data_per_user[j])

            if len(common) in list(intersection_per_user.keys()) :
                intersection_per_user[len(common)].append((i, j))
            else :
                intersection_per_user[len(common)] = [(i, j)]
            """
            similarity = get_similarity(data_per_user[i], data_per_user[j])
            if count % 1000 == 0 :
                print(i, j, similarity)
            count = count + 1
            if (similarity) in list(intersection_per_user.keys()) :
                intersection_per_user[similarity].append((i, j))
            else :
                intersection_per_user[similarity] = [(i, j)]

    intersection_per_user = collections.OrderedDict(sorted(intersection_per_user.items()))
    neighbor_dict = {}
    neighbor_list = []

    for i in users :
        user_neighbor = []
        for j in intersection_per_user.keys() :
            if j > 0 :
                for k in intersection_per_user[j] :
                    if i in k :
                        user_neighbor.append([j, list(set(k)-set([i]))[0]])

        neighbor_list.append(user_neighbor)
        neighbor_dict[i] = neighbor_list

    return neighbor_dict

In [15]:
def get_common_item (item_x, item_y) :
    common_item = []
    for i in item_x :
        for j in item_y :
            if get_similarity(i, j) :
                common_item.append(i)
    return common_item

In [16]:
def get_similarity (item_x, item_y) :
    item_x_rank = list(map(lambda x: x[1], item_x))
    item_y_rank = list(map(lambda y: y[1], item_y))
    avg_x = sum(item_x_rank) / len(item_x_rank)
    avg_y = sum(item_y_rank) / len(item_y_rank)
    x_square_root = sum(map(lambda x : (x - avg_x)**2, item_x_rank))**0.5
    y_square_root = sum(map(lambda y : (y - avg_y)**2, item_y_rank))**0.5
    similarity = sum([(ix[1]-avg_x)*(iy[1]-avg_y) for ix, iy in zip(item_x, item_y) if ix[0] == iy[0]])
    return ((similarity) / (x_square_root * y_square_root))

In [17]:
neighbor = get_neighbor(data_set_per_user)

1 2 0.002873608728462925
2 61 0.0
3 121 0.0
4 182 0.0
5 244 0.0
6 307 0.006610067403160352
7 371 0.0
8 436 -0.01181477712414642
9 502 0.0
10 569 0.0
11 637 0.0
12 706 0.0
13 776 0.0
14 847 0.0
15 919 0.0
17 66 0.0
18 141 0.0
19 217 0.0
20 294 -0.008710539225272594
21 372 -0.09116676652511908
22 451 0.0
23 531 0.0
24 612 0.0
25 694 0.0
26 777 -9.076595857898872e-05
27 861 0.0
29 32 0.0
30 119 0.0
31 207 0.0
32 296 0.0
33 386 0.0
34 477 0.0
35 569 0.0
36 662 0.0
37 756 0.0
38 851 0.0
40 44 0.0
41 142 0.0
42 241 0.0
43 341 0.0
44 442 0.013097617038670293
45 544 0.0
46 647 0.0
47 751 0.0
48 856 0.0
50 69 0.0
51 177 0.0
52 286 0.0
53 396 0.0
54 507 0.0
55 619 0.0
56 732 0.0
57 846 0.0
59 77 -0.026909675124445784
60 194 0.0
61 312 0.0
62 431 0.0
63 551 -0.0076490994766197
64 672 0.0
65 794 0.041552085141904976
66 917 0.0
68 166 0.0
69 292 0.0
70 419 -0.0030097541412869833
71 547 0.0
72 676 0.0026793467120729635
73 806 -0.007460833638532521
74 937 0.0
76 202 0.0
77 336 -0.00019270902605606464

In [18]:
# test result print
with open(test_file) as test :
    test_set = test.read().split()
    test_set = list(map(int, test_set))
    test_set = [test_set[i:i+4] for i in range(0, len(test_set), 4)]
test_set = list(map(lambda x : [x[0], x[1]], test_set))

In [19]:
def predict (neighbor, data_per_user, test_set) :
    neighbor_rank = []
    
    for i in neighbor[test_set[0]] :
        for j in i :
            for k in data_per_user[j[1]] :
                if k[0] == test_set[1] :
                    neighbor_rank.append([k[1], j[0]])
            
    if len(neighbor_rank) > 0 :
        return round(sum(map(lambda x : x[0]*(x[1]), neighbor_rank))/sum(map(lambda x : (x[1]), neighbor_rank)))
    else :
        rank_list = list(map(lambda x: x[1], data_per_user[test_set[0]]))
        return max(set(rank_list), key=rank_list.count)

In [ ]:
count = 0
with open(output_file, "w") as prediction :
    for i in test_set :
        if count % 10 == 0 :
            print(count)
        count = count+1
        prediction.write("{}\t{}\t{}\n".format(i[0], i[1], str(predict(neighbor, data_set_per_user, i))))

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
